# A61H CPC Classification - A61H0700

<font size="3">

#### About the Module:
<I><span style="font-family:Arial">This module work to identify patents in A61H700 class. Class detail is as follow<br></span>
<span style="font-family:Arial">A61H7/00&emsp; &emsp; &emsp; &emsp;•&emsp;Devices for suction-kneading massage; Devices for massaging the skin by rubbing or brushing<br></span>
<span style="font-family:Arial">A61H7/001 &emsp; &emsp; &emsp; •&emsp;without substantial movement between the skin and the device<br></span>
<span style="font-family:Arial">A61H7/002 &emsp; &emsp; &emsp; •&emsp;by rubbing or brushing<br></span>
<span style="font-family:Arial">A61H7/003 &emsp; &emsp; &emsp; ••&emsp;Hand-held or hand-driven devices<br></span>
<span style="font-family:Arial">A61H7/004 &emsp; &emsp; &emsp; ••&emsp;power-driven, e.g. electrical<br></span>
<span style="font-family:Arial">A61H7/005 &emsp; &emsp; &emsp; •••&emsp;hand-held<br></span>
<span style="font-family:Arial">A61H7/006 &emsp; &emsp; &emsp; •&emsp;Helmets for head-massage<br></span>
<span style="font-family:Arial">A61H7/007 &emsp; &emsp; &emsp; •&emsp;Kneading<br></span>
<span style="font-family:Arial">A61H7/008 &emsp; &emsp; &emsp; ••&emsp;Suction kneading<br></span>
<span style="font-family:Arial">A61H2007/009 &emsp;&emsp; ••&emsp;having massage elements rotating on parallel output axis<br></span>

#### Input needed: 
<I><span style="font-family:Arial">This module use libraries, function and processed text data present in initial_processing module</span></I>

#### Output expected: 
<I><span style="font-family:Arial">This module will identify patents in A61H700 class in test data.<br>This output is pass to the module final_file. No output is expected in this module alone.</span></I>

#### Related modules: 
<I><span style="font-family:Arial">This module is calling initial_processing module and is called by final_file module</span></I>

#### Who and when: 
<I><span style="font-family:Arial">Last Modified by : Nishant Chauhan</span><br>
<span style="font-family:Arial">Last Modified on : 27-July-2020</span><br>
<span style="font-family:Arial">Version no       : 2</span><br>
<span style="font-family:Arial">Developed by     : Nishant Chauhan </span><br></font></I>


In [1]:
import os
import sys
import import_ipynb
from Initial_processing import *

importing Jupyter notebook from Initial_processing.ipynb
Module Initial Processing is successfully loaded on 01/Aug/2020 - 23:59 PM


In [2]:
df['A61H0700'] = df['CPC'].str.contains('A61H7/|A61H0007|A61H 7/').astype('int')
df['A61H0700'].value_counts()

0    3473
1     444
Name: A61H0700, dtype: int64

## Model training and prediction

In [3]:
y_train = df['A61H0700']

model_title , score_title = train_model(X_title_dtm, y_train)

y_pred_class_title = model_title.predict(X_title_test_dtm)

In [4]:
model_abstract , score_abstract = train_model(X_abstract_dtm, y_train)

y_pred_class_abstract = model_abstract.predict(X_abstract_test_dtm)

In [5]:
model_tab , score_tab = train_model(X_tab_dtm, y_train)

y_pred_class_tab = model_tab.predict(X_tab_test_dtm)

print("Training data accuracy score : " ,((score_abstract.mean() + score_title.mean() + score_tab.mean())/3))

Training data accuracy score :  0.898481055201802


In [6]:
df_first = pd.DataFrame(df_test[['number','title','abstract']])

In [7]:
df_first['all_text'] = X_all
df_first['title_prediction'] = y_pred_class_title
df_first['abstract_prediction'] = y_pred_class_abstract

## Predicting using words

In [152]:
# AND Operator keywords

A61H0700 = ['kneading', 'suction knead','knead massag']


In [153]:
# NEAR operator keywords (List1 NEAR5 List2)

A61H0700_list1 = ['knead','kneading','rubbing', 'brushing']
A61H0700_list2 = ['therapy', 'massag']

A61H0700_list3 = ['helmet','headset']
A61H0700_list4 = ['head','scalp','forehead']
A61H0700_list5 = ['massag']



In [154]:
#SSTO command e.g. search "eye massage device" as single word

A61H0700_word = ['kneading massag','rubbing massag','brushing massag','skin massag']

In [155]:
new_col = []
for i in range(len(df_first['all_text'])):
    check_word = (near_operator(df_first['all_text'][i],A61H0700_list1,A61H0700_list2, near=5) or
                  (near_operator(df_first['all_text'][i],A61H0700_list3,A61H0700_list4, near=5) and 
                  near_operator(df_first['all_text'][i],A61H0700_list4,A61H0700_list5, near=5)) or
                    and_operator(df_first['all_text'][i],A61H0700) or
                     ssto(df_first['all_text'][i], A61H0700_word))
                                 
    new_col.append(check_word)
    
df_first['word_predict'] = new_col

In [156]:
df_first['A61H0700'] = df_first['word_predict'] + df_first['title_prediction'] + df_first['abstract_prediction']

for row in range(len(df_first['A61H0700'])):
    if df_first.loc[row, 'A61H0700'] > 0:
        df_first.loc[row, 'A61H0700'] = 1
    else:
        df_first.loc[row, 'A61H0700'] = 0

In [157]:
df_second = df_first.drop(['title_prediction','abstract_prediction', 'word_predict'],axis=1)

## Sub-classification of A61H7/00 (this section need improvement)

### A61H7/001       • without substantial movement between the skin and the device

In [158]:
# AND Operator keywords
A61H7001 = ['without movement between skin device']

# NEAR operator keywords (List1 NEAR5 List2)
A61H7001_list1 = []
A61H7001_list2 = []

#SSTO command e.g. search "eye massage device" as single word
A61H7001_word = []

In [159]:
new_col_7001 = []
for i in range(len(df_second['all_text'])):
    if df_second['A61H0700'][i] == 1:
        check_word_7001 = int((near_operator(df_second['all_text'][i],A61H7001_list1,A61H7001_list2, near=5) or 
                           and_operator(df_second['all_text'][i],A61H7001) or
                           ssto(df_second['all_text'][i], A61H7001_word)))
    else:
        check_word_7001 = 0
    
    new_col_7001.append(check_word_7001)
    
df_second['A61H7001'] = new_col_7001

### A61H7/002       • by rubbing or brushing

In [160]:
# AND Operator keywords
A61H7002 = []

# NEAR operator keywords (List1 NEAR5 List2)
A61H7002_list1 = ['rubbing','brushing']
A61H7002_list2 = ['massag','therap']

#SSTO command e.g. search "eye massage device" as single word
A61H7002_word = []

In [161]:
new_col_7002 = []
for i in range(len(df_second['all_text'])):
    if df_second['A61H0700'][i] == 1:
        check_word_7002 = int((near_operator(df_second['all_text'][i],A61H7002_list1,A61H7002_list2, near=5) or 
                           and_operator(df_second['all_text'][i],A61H7002) or
                           ssto(df_second['all_text'][i], A61H7002_word)))
    else:
        check_word_7002 = 0
    
    new_col_7002.append(check_word_7002)
    
df_second['A61H7002'] = new_col_7002

### A61H7/004       •• power-driven, e.g. electrical

In [162]:
# AND Operator keywords
A61H7004 = []

# NEAR operator keywords (List1 NEAR5 List2)
A61H7004_list1 = ['driven','operate']
A61H7004_list2 = ['electric','power', 'motor']

#SSTO command e.g. search "eye massage device" as single word
A61H7004_word = []

In [163]:
new_col_7004 = []
for i in range(len(df_second['all_text'])):
    if df_second['A61H0700'][i] == 1:
        check_word_7004 = int((near_operator(df_second['all_text'][i],A61H7004_list1,A61H7004_list2, near=5) or 
                           and_operator(df_second['all_text'][i],A61H7004) or
                           ssto(df_second['all_text'][i], A61H7004_word)))
    else:
        check_word_7004 = 0
    
    new_col_7004.append(check_word_7004)
    
df_second['A61H7004'] = new_col_7004

### A61H7/005       ••• hand-held

In [164]:
# AND Operator keywords
A61H7005 = []

# NEAR operator keywords (List1 NEAR5 List2)
A61H7005_list1 = ['handheld']
A61H7005_list2 = ['device','apparatus']

#SSTO command e.g. search "eye massage device" as single word
A61H7005_word = ['hand driven', 'hand operat']

In [165]:
new_col_7005 = []
for i in range(len(df_second['all_text'])):
    if df_second['A61H7004'][i] == 1:
        check_word_7005 = int((near_operator(df_second['all_text'][i],A61H7005_list1,A61H7005_list2, near=5) or 
                           and_operator(df_second['all_text'][i],A61H7005) or
                           ssto(df_second['all_text'][i], A61H7005_word)))
    else:
        check_word_7005 = 0
    
    new_col_7005.append(check_word_7005)
    
df_second['A61H7005'] = new_col_7005

### A61H7/003       •• Hand-held or hand-driven devices

In [166]:
# AND Operator keywords
A61H7003 = []

# NEAR operator keywords (List1 NEAR5 List2)
A61H7003_list1 = ['handheld']
A61H7003_list2 = ['device','apparatus']

#SSTO command e.g. search "eye massage device" as single word
A61H7003_word = ['hand driven', 'hand operat']

In [167]:
new_col_7003 = []
for i in range(len(df_second['all_text'])):
    if df_second['A61H0700'][i] == 1 & df_second['A61H7004'][i] == 0:
        check_word_7003 = int((near_operator(df_second['all_text'][i],A61H7003_list1,A61H7003_list2, near=5) or 
                           and_operator(df_second['all_text'][i],A61H7003) or
                           ssto(df_second['all_text'][i], A61H7003_word)))
    else:
        check_word_7003 = 0
    
    new_col_7003.append(check_word_7003)
    
df_second['A61H7003'] = new_col_7003

### A61H7/006       • Helmets for head-massage

In [168]:
# AND Operator keywords
A61H7006 = []

# NEAR operator keywords (List1 NEAR5 List2)
A61H7006_list1 = ['helmet','headgear','headset']
A61H7006_list2 = ['head','scalp','forehead']
A61H7006_list3 = ['massag']

#SSTO command e.g. search "eye massage device" as single word
A61H7006_word = []

In [169]:
new_col_7006 = []
for i in range(len(df_second['all_text'])):
    if df_second['A61H0700'][i] == 1:
        check_word_7006 = int(((near_operator(df_second['all_text'][i],A61H7006_list1,A61H7006_list2, near=7) and
                               near_operator(df_second['all_text'][i],A61H7006_list2,A61H7006_list3, near=7)) or
                           and_operator(df_second['all_text'][i],A61H7006) or
                           ssto(df_second['all_text'][i], A61H7006_word)))
    else:
        check_word_7006 = 0
    
    new_col_7006.append(check_word_7006)
    
df_second['A61H7006'] = new_col_7006

### A61H7/007       • Kneading

In [170]:
# AND Operator keywords
A61H7007 = ['knead']

# NEAR operator keywords (List1 NEAR5 List2)
A61H7007_list1 = []
A61H7007_list2 = []

#SSTO command e.g. search "eye massage device" as single word
A61H7007_word = []

In [171]:
new_col_7007 = []
for i in range(len(df_second['all_text'])):
    if df_second['A61H0700'][i] == 1:
        check_word_7007 = int((near_operator(df_second['all_text'][i],A61H7007_list1,A61H7007_list2, near=5) or 
                           and_operator(df_second['all_text'][i],A61H7007) or
                           ssto(df_second['all_text'][i], A61H7007_word)))
    else:
        check_word_7007 = 0
    
    new_col_7007.append(check_word_7007)
    
df_second['A61H7007'] = new_col_7007

### A61H7/008       •• Suction kneading

In [172]:
# AND Operator keywords
A61H7008 = []

# NEAR operator keywords (List1 NEAR5 List2)
A61H7008_list1 = ['suction']
A61H7008_list2 = ['knead']

#SSTO command e.g. search "eye massage device" as single word
A61H7008_word = []

In [173]:
new_col_7008 = []
for i in range(len(df_second['all_text'])):
    if df_second['A61H7007'][i] == 1:
        check_word_7008 = int((near_operator(df_second['all_text'][i],A61H7008_list1,A61H7008_list2, near=5) or 
                           and_operator(df_second['all_text'][i],A61H7008) or
                           ssto(df_second['all_text'][i], A61H7008_word)))
    else:
        check_word_7008 = 0
    
    new_col_7008.append(check_word_7008)
    
df_second['A61H7008'] = new_col_7008

### A61H2007/009    •• having massage elements rotating on parallel output axis

In [185]:
# AND Operator keywords
A61H2007009 = []

# NEAR operator keywords (List1 NEAR5 List2)
A61H2007009_list1 = ['massage', 'massaging','massager']
A61H2007009_list2 = ['rotat','revolv' ]
A61H2007009_list2 = ['parallel axis','output axis', 'output shaft']


#SSTO command e.g. search "eye massage device" as single word
A61H2007009_word = []

In [186]:
new_col_2007009 = []
for i in range(len(df_second['all_text'])):
    if df_second['A61H7007'][i] == 1:
        check_word_2007009 = int(((near_operator(df_second['all_text'][i],A61H2007009_list1,A61H2007009_list2, near=5) and
                                  near_operator(df_second['all_text'][i],A61H2007009_list1,A61H2007009_list3, near=10)) or
                           and_operator(df_second['all_text'][i],A61H2007009) or
                           ssto(df_second['all_text'][i], A61H2007009_word)))
    else:
        check_word_2007009 = 0
    
    new_col_2007009.append(check_word_2007009)
    
df_second['A61H2007009'] = new_col_2007009

## Final step

In [187]:
for i in range(4,len(df_second.columns)):
    df_second.iloc[:,i].replace(1,df_second.columns[i], inplace=True)
    df_second.iloc[:,i].replace(0,'', inplace=True)

In [188]:
cols = ['A61H0700', 'A61H7001','A61H7002', 'A61H7004', 'A61H7005', 'A61H7003',
        'A61H7006', 'A61H7007','A61H7008', 'A61H2007009']

df_second['prediction'] = ''
for i in range(len(df_second)):
    for col in cols:
        if not df_second.loc[i,col] == '':
            df_second.loc[i,'prediction'] = (df_second.loc[i,'prediction'] + ' | '
                                                        + df_second.loc[i,col])

df_second['prediction'] = df_second['prediction'].str[3:]

In [189]:
df_final = df_second.drop(columns=['A61H0700', 'A61H7001','A61H7002', 'A61H7004', 'A61H7005', 'A61H7003',
        'A61H7006', 'A61H7007','A61H7008', 'A61H2007009', 'all_text'], axis=1)


In [190]:
from datetime import datetime
dt_string = datetime.now().strftime("%d/%b/%Y - %H:%M %p")
print("Module A61H0700 is successfully loaded on",dt_string)

Module A61H0700 is successfully loaded on 02/Aug/2020 - 02:00 AM
